# Машинное обучение

## Факультет математики НИУ ВШЭ

### 2020-2021 учебный год

Лектор: Илья Щуров

Семинаристы: Руслан Хайдуров, Соня Дымченко

Ассистенты: Максим Бекетов, Павел Егоров

# Семинар 7

Сегодня мы:

- познакомимся с автоматической обработкой текстов
- рассмотрим базовые методы для работы с текстами
- решим задачу анализа тональности текстов благодаря полученным навыкам!

### Зачем?

Примеры задач автоматической обработки текстов:

- классификация текстов

    - анализ тональности
    - фильтрация спама
    - по теме или жанру

- машинный перевод

- распознавание речи

- извлечение информации

    - именованные сущности
    - факты и события

- кластеризация текстов

- оптическое распознавание символов

- проверка правописания

- вопросно-ответные системы

- суммаризация текстов

- генерация текстов

Сегодня рассмотрим следующие приемы для работы с текстами:

**Предобработка текста**
- токенизация
- лемматизация / стемминг
- удаление стоп-слов

**Векторные представления текстов**

- bag of words
- TF-IDF

In [91]:
import numpy as np
import pandas as pd

# Предобработка текста

## 1 Токенизация

Чтобы качественно проанализировать текст, нужно разграничить в нем токены - отдельные слова. Процесс такого разбиения называется ***токенизацией***. В данный метод также может включаться избавление от знаков препинания и преобразование заглавных букв в строчные.

Рассмотрим небольшой пример. Предположим, нужно проделать токенизацию первых двух предложений из романа Фрэнсиса Скотта Фицджеральда "Великий Гэтсби":

In [1]:
text = '''In my younger and more vulnerable years my father gave me some advice that \
I've been turning over in my mind ever since.\n\"Whenever you feel like criticizing any one,\
\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\"'''
print(text)

In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since.
"Whenever you feel like criticizing any one," he told me, "just remember that all the people in this world haven't had the advantages that you've had."


Попробуем разделить текст по пробелам:

In [2]:
print(text.split())

['In', 'my', 'younger', 'and', 'more', 'vulnerable', 'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that', "I've", 'been', 'turning', 'over', 'in', 'my', 'mind', 'ever', 'since.', '"Whenever', 'you', 'feel', 'like', 'criticizing', 'any', 'one,"', 'he', 'told', 'me,', '"just', 'remember', 'that', 'all', 'the', 'people', 'in', 'this', 'world', "haven't", 'had', 'the', 'advantages', 'that', "you've", 'had."']


In [3]:
# import re
# re.split("[,\t\n\ \".]+", text.lower())

Видно, что вместе со словами выделились и знаки пунктуации. Можно просто убрать их вручную (или воспользовать регулярными выражениями), однако реальные *данные наполнены различным шумом* (html-разметка, ссылки, лишние знаки пунктуации) и опечатками, что создает дополнительные трудности.

Во избежание вышеупомянутых проблем, для токенизации используются морфологические правила, основанные на регулярных выражениях.

In [4]:
# !pip3 install nltk

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/svdcvt/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [32]:
from nltk.tokenize import word_tokenize
print(word_tokenize(text.lower()))

['in', 'my', 'younger', 'and', 'more', 'vulnerable', 'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that', 'i', "'ve", 'been', 'turning', 'over', 'in', 'my', 'mind', 'ever', 'since', '.', '``', 'whenever', 'you', 'feel', 'like', 'criticizing', 'any', 'one', ',', "''", 'he', 'told', 'me', ',', '``', 'just', 'remember', 'that', 'all', 'the', 'people', 'in', 'this', 'world', 'have', "n't", 'had', 'the', 'advantages', 'that', 'you', "'ve", 'had', '.', "''"]


Можно оставить токены, содержащие только буквы:

In [34]:
text_tokenized = [w for w in word_tokenize(text.lower()) if w.isalpha()]
print(text_tokenized)

['in', 'my', 'younger', 'and', 'more', 'vulnerable', 'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that', 'i', 'been', 'turning', 'over', 'in', 'my', 'mind', 'ever', 'since', 'whenever', 'you', 'feel', 'like', 'criticizing', 'any', 'one', 'he', 'told', 'me', 'just', 'remember', 'that', 'all', 'the', 'people', 'in', 'this', 'world', 'have', 'had', 'the', 'advantages', 'that', 'you', 'had']


## 2 Лемматизация и стемминг

После токенизации можно применить лемматизацию и/или стемминг.

**Лемматизация** - процедура, при которой все выделенные словоформы _приводятся к своим леммам_ (нормальным формам).

> Например, токены "пью", "пил", "пьет" перейдут в "пить".  

Здесь, как и в токенизации, возникают неопределенности, связанные с зависимостью смыслов слов от контекста: например, "рой" может быть глаголом в повелительном наклонении, образованным от глагола "рыть", или же существительным в именительном падеже ("пчелиный рой"). Неопределенности можно разрешить с помощью вероятностной модели, которая будет рассматривать контекст (слова, расположенные рядом с данным) и определять, с какой вероятностью данное слово имеет тот или иной смысл.

При применении **стемминга** у всех слов _отбрасываются аффиксы_ (окончания и суффиксы). 

> Например, токены "идет", "идущий", "идя", "идут" перейдет в "ид".

То, что осталось от слова по окончании процедуры, называют **стемом**. Приводя различные синтетические формы одного и того же слова к одному виду, стемминг существенно может улучшить качество модели. Однако здесь тоже встречаются неопределенности: например, "белка", "белый" и "белье" при тривиальном стемминге переходят в "бел", "скорый" и "поскорее" переходят в разные стемы "скор" и "поскор". Эти неопределенности можно разрешить путем последовательного применения ряда морфологических правил.

#### Зачем это делать?

1. Уменьшение размера словаря
2. Это может быть необходимо в некоторых задачах, например
    - Создание классов эквивалентности в __информационном поиске__:
        - кошка, кошки, кошку, кошкой, кошке… -> кошка

In [35]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/svdcvt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

In [37]:
print(*text_tokenized, sep=' ')

in my younger and more vulnerable years my father gave me some advice that i been turning over in my mind ever since whenever you feel like criticizing any one he told me just remember that all the people in this world have had the advantages that you had


In [38]:
text_stemmed = [stemmer.stem(w) for w in text_tokenized]
print(*text_stemmed, sep=' ')

in my younger and more vulner year my father gave me some advic that i been turn over in my mind ever sinc whenev you feel like critic ani one he told me just rememb that all the peopl in this world have had the advantag that you had


In [39]:
text_lemmatized = [lemmatizer.lemmatize(w) for w in text_tokenized]
print(*text_lemmatized, sep=' ')

in my younger and more vulnerable year my father gave me some advice that i been turning over in my mind ever since whenever you feel like criticizing any one he told me just remember that all the people in this world have had the advantage that you had


```
years - year - year
criticizing - critic - criticizing
any - ani - any
remember - rememb - remember
advantages - advantag - advantage
```

### Особенности работы с русскими текстами

1. Используется в основном лемматизация, а не стемминг, потому что хорошего стеммера для руссого нет (Snowball тоже работает с русским, но обработка лемматизацией работает лучше)

2. Для лемматизации используется либо [PyMorphy](https://nlpub.ru/Pymorphy), либо [MyStem](https://nlpub.ru/Mystem)


## 3 Стоп-слова

В тексте могут встречаться слова, не несущие в себе абсолютно никакой информации - шумовые, или стоп-слова. Их можно отфильтровать.

In [40]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/svdcvt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
stop_words = set(stopwords.words('english'))

In [42]:
text_stemmed_stopped = [w for w in text_stemmed if w not in stop_words]
print(*text_stemmed_stopped, sep=' ')

younger vulner year father gave advic turn mind ever sinc whenev feel like critic ani one told rememb peopl world advantag


In [43]:
text_lemmatized_stopped = [w for w in text_lemmatized if w not in stop_words]
print(*text_lemmatized_stopped, sep=' ')

younger vulnerable year father gave advice turning mind ever since whenever feel like criticizing one told remember people world advantage


In [44]:
print(*[w.lower() for w in text_lemmatized if w.lower() not in stop_words], sep=' ')

younger vulnerable year father gave advice turning mind ever since whenever feel like criticizing one told remember people world advantage


# Векторные представления текстов

Но как же все-таки работать с текстами, используя стандартные методы машинного обучения? Нужна выборка!

## Bag-of-words

Текст можно представить как набор независимых слов. Тогда каждому слову можно сопоставить вес, таким образом, сопоставляя тексту набор весов. В качестве весов можно брать частоту встречаемости слов в тексте.

In [45]:
texts = ['I like my cat.', 'My cat is the most perfect cat.', 'is this cat or is this bread?']

In [79]:
texts_tokenized = [' '.join([w for w in word_tokenize(t) if w.isalpha()]) for t in texts]
texts_tokenized

['I like my cat',
 'My cat is the most perfect cat',
 'is this cat or is this bread']

In [75]:
texts_tokenized = [' '.join([stemmer.stem(w) for w in word_tokenize(t) if w.isalpha()]) for t in texts]
texts_tokenized

['i like my cat',
 'my cat is the most perfect cat',
 'is this cat or is this bread']

In [59]:
texts_tokenized = [' '.join([stemmer.stem(w) for w in word_tokenize(t) if w.isalpha() and w not in stop_words]) for t in texts]
texts_tokenized

['i like cat', 'my cat perfect cat', 'cat bread']

Как обычно, выручает `sklearn`:

In [80]:
from sklearn.feature_extraction.text import CountVectorizer

# метод подсчитывает количество слов всего и подсчитывает веса для каждого примера (предложения/текста)
cnt_vec = CountVectorizer()
X = cnt_vec.fit_transform(texts_tokenized) # сжатая матрица весов (sparse matrix, потому что очень много нулей)

In [81]:
type(X), X.shape

(scipy.sparse.csr.csr_matrix, (3, 10))

In [82]:
cnt_vec.get_feature_names(), len(cnt_vec.get_feature_names())

(['bread', 'cat', 'is', 'like', 'most', 'my', 'or', 'perfect', 'the', 'this'],
 10)

In [70]:
X.toarray()

array([[0, 1, 1, 0, 0],
       [0, 2, 0, 1, 1],
       [1, 1, 0, 0, 0]])

## TF-IDF

Заметим, что если слово часто встречается в одном тексте, но почти не встречается в других, то оно получает для данного текста большой вес, ровно так же, как и слова, которые часто встречаются в каждом тексте. 
Для того, чтобы разделять такие слова - популярные среди текстов в принципе и популярные-уникальные для текста, можно использовать **статистическую меру TF-IDF, характеризующую важность слова для конкретного текста**. 

D_1 = 'like my cat'

D_2 = 'my cat is the most perfect cat'

D_3 = 'is this cat or is this bread'

word |frequency in D1| D2 | D3
---  | ---           | ---| ---
bread| 0             | 0 | 1
cat  | 1             | 2 | 1
is   | 0 | 1 | 2
like | 1 | 0 |0 
most | 0 | 1 | 0
my   | 1 | 1 | 0
or   | 0 | 0 | 1
perfect | 0 | 1 | 0
the  | 0 | 1 | 0
this | 0 | 0 | 2
---- | --- | --- | --- |
sum()| 3 | 7 | 7

Для каждого слова $t$ из текста $d$ рассчитаем относительную частоту встречаемости в нем (Term Frequency):

$$
\text{TF}(t, d) = \frac{C(t)}{\sum\limits_{k \in d}C(k)} = \frac{\text{число вхождений слова $t$ в текст $d$}}{\text{количество всех слов в тексте $d$}},
$$

Также для каждого слова из текста $d$ рассчитаем обратную частоту встречаемости в корпусе текстов $D$ (Inverse Document Frequency):

$$
\text{IDF}(t, D) = \log\left(\frac{|D|}{|\{d_i \in D \mid t \in d_i\}|}\right) = \log(\frac{\text{общее число документов}}{\text{число документов, куда входит слово $t$}})
$$

Логарифмирование здесь проводится с целью уменьшить масштаб весов, ибо зачастую в корпусах присутствует очень много текстов.

В итоге каждому слову $t$ из текста $d$ теперь можно присвоить вес

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Интерпретировать формулу выше несложно: действительно, чем чаще данное слово встречается в данном тексте и чем реже в остальных, тем важнее оно для этого текста.

**А что там с практикой? `sklearn`, на помощь!**

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer

# метод возвращает матрицу с весами для слов для каждого примера (текста/предложения)
tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(texts_tokenized)

In [89]:
type(X), X.shape

(scipy.sparse.csr.csr_matrix, (3, 10))

In [84]:
tfidf_vec.get_feature_names()

['bread', 'cat', 'is', 'like', 'most', 'my', 'or', 'perfect', 'the', 'this']

In [90]:
X.toarray()

array([[0.        , 0.42544054, 0.        , 0.72033345, 0.        ,
        0.54783215, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.50130994, 0.32276391, 0.        , 0.42439575,
        0.32276391, 0.        , 0.42439575, 0.42439575, 0.        ],
       [0.33976626, 0.20067143, 0.516802  , 0.        , 0.        ,
        0.        , 0.33976626, 0.        , 0.        , 0.67953252]])

Что изменилось по сравнению с методом `CountVectorizer`? Интерпретируйте результат.

# Практика

Вам предлагается решить задачу анализа тональности - построить модель, определяющую по отзыву о фильме, положительный он или отрицательный.

In [92]:
try:
    data = pd.read_csv('movie_reviews.csv')
except:
    data = pd.read_csv('https://raw.githubusercontent.com/ischurov/math-ml-hse-2018/master/sem08_texts/movie_reviews.csv')

In [46]:
# your code here

In [93]:
data.head()

,review,positive
0,"tristar / 1 : 30 / 1997 / r ( language , viole...",0
1,arlington road 1/4 . directed by mark pellingt...,0
2,the brady bunch movie is less a motion picture...,0
3,janeane garofalo in a romantic comedy -- it wa...,0
4,"i'm going to keep this plot summary brief , so...",0


In [80]:
# 1) токенизировать

,review,positive
0,"[tristar, /, 1, :, 30, /, 1997, /, r, (, langu...",0
1,"[arlington, road, 1/4, ., direct, mark, pellin...",0
2,"[bradi, bunch, movi, le, motion, pictur, minor...",0
3,"[janean, garofalo, romant, comedi, --, good, i...",0
4,"['m, go, keep, plot, summari, brief, ,, someth...",0
...,...,...
1395,"[one, last, entri, long-run, carri, seri, ,, c...",1
1396,"[hype, ?, sheesh, ,, like, ., side, titan, ,, ...",1
1397,"[u, n't, yet, born, 1960, 's, rock, ', n, ', r...",1
1398,"[start, monoton, talking-head, music, histori,...",1


In [81]:
# 2) лемматизировать или стемматизировать

In [94]:
# 3) TfidfVectorizer() или CountVectorizer()

In [95]:
# 4) в итоге получить матрицу X_vec

In [ ]:
# 5) разделить выбору на трейн-тест

In [96]:
# 6) обучить классификатор из sklearn 

In [ ]:
# 7) посчитать метрики для каждого классификатора и сравнить

In [ ]:
# (*) посмотреть какой вес дает logreg словам (самый большой и самый маленький)